In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl
import pandas as pd
import seaborn as sns
import IPython
import numpy as np
import sweetviz as sv

import sklearn
from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from catboost              import CatBoostRegressor

from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier


set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24









Pandas   1.2.5
Sklearn  0.24.2


In [2]:
import glob
col = ["X", "Y", "Z", "Pressure", "GripAngle", "Timestamp", "Test ID"]
control_data = pd.DataFrame()
control_file = glob.glob("./control/*.txt")

for f in control_file:
    data = pd.read_csv(f, delimiter=";")
    data.columns = col
    # print(data)
    control_data = pd.concat([control_data, data], axis=0)

print(control_data.shape)

#creating the class column
control_data["Class"] = 0

control_data


(120295, 7)


,X,Y,Z,Pressure,GripAngle,Timestamp,Test ID,Class
0,200,204,0,218,900,1732647307,0,0
1,200,204,0,253,900,1732647314,0,0
2,200,204,0,304,900,1732647321,0,0
3,200,204,0,351,900,1732647328,0,0
4,200,204,0,386,900,1732647335,0,0
...,...,...,...,...,...,...,...,...
6671,242,242,272,0,1530,6562237,2,0
6672,242,242,272,0,1460,6562245,2,0
6673,242,242,272,0,1460,6562252,2,0
6674,242,242,272,0,1460,6562259,2,0


In [3]:
col = ["X", "Y", "Z", "Pressure", "GripAngle", "Timestamp", "Test ID"]
parkinson_data = pd.DataFrame()
parkinson_file = glob.glob("./parkinson/*.txt")

for f in parkinson_file:
    data = pd.read_csv(f, delimiter=";")
    data.columns = col
    # print(data)
    parkinson_data = pd.concat([parkinson_data, data], axis=0)

print(parkinson_data.shape)

#creating the class column
parkinson_data["Class"] = 1


(404371, 7)


Concat data

DATA  AGUMENTATION

In [4]:
df = pd.concat([parkinson_data, control_data], axis=0)
print(df)

extra_sample = df.sample(df.shape[0] // 3 )

df_extra = pd.concat([df, extra_sample], axis=0)
df_extra.drop('Timestamp', axis=1, inplace = True)
df_extra

        X    Y    Z  Pressure  GripAngle  Timestamp  Test ID  Class
0     201  200    0        86        900   68140154        0      1
1     201  200    0       123        900   68140161        0      1
2     201  200    0       146        900   68140168        0      1
3     201  200    0       158        900   68140175        0      1
4     201  200    0       159        900   68140182        0      1
...   ...  ...  ...       ...        ...        ...      ...    ...
6671  242  242  272         0       1530    6562237        2      0
6672  242  242  272         0       1460    6562245        2      0
6673  242  242  272         0       1460    6562252        2      0
6674  242  242  272         0       1460    6562259        2      0
6675  242  242  260         0       1350    6562266        2      0

[524666 rows x 8 columns]


,X,Y,Z,Pressure,GripAngle,Test ID,Class
0,201,200,0,86,900,0,1
1,201,200,0,123,900,0,1
2,201,200,0,146,900,0,1
3,201,200,0,158,900,0,1
4,201,200,0,159,900,0,1
...,...,...,...,...,...,...,...
12088,46,179,0,830,940,1,1
5518,260,159,0,938,940,2,1
4555,251,231,0,426,1410,2,1
16051,197,233,0,803,1820,2,1


In [5]:
std_pressure = df_extra['Pressure'].std()/10

df_extra['Enh_pressure'] = df_extra['Pressure'] + std_pressure



In [6]:
y = df_extra['Pressure']
x = df_extra.drop('Pressure', axis=1)


In [7]:
y 

0         86
1        123
2        146
3        158
4        159
        ... 
12088    830
5518     938
4555     426
16051    803
3921     769
Name: Pressure, Length: 699554, dtype: int64

In [14]:
scaler = StandardScaler()

x = scaler.fit_transform(x)

y = np.array(y)
y = y.reshape(-1,1)
y =scaler.fit_transform(y)
print(y.shape)
print(x.shape)


(699554, 1)
(699554, 7)


In [10]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":   ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  #"AdaBoost":      AdaBoostClassifier(),
 # "Skl GBM":       GradientBoostingClassifier(),
  #"Skl HistGBM":   GradientBoostingClassifier(),
  #"XGBoost":       XGBClassifier(),
  #"LightGBM":      LGBMClassifier(),
  #"CatBoost":      CatBoostClassifier() 
}

In [11]:
# CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y,test_size = 0.2,stratify = y,random_state = 42)

# Your code goes here


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items(): # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    start_time = time.time()
    model.fit(x_train, y_train)
    pred = model.predict(x_val)    # GET PREDICTIONS USING x_val
    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# Your code goes here


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

ValueError: Unknown label type: 'continuous'

In [ ]:
##########################################################################################################################################

JONS CODE

In [4]:
import glob
parkinson_path = "./parkinson/*.txt"
control_path = "./control/*.txt"

def  getFiles(path):
    all_files = glob.glob(path)
    data = []
    
    for file in all_files:
        df = pd.read_csv(file, sep= ';')
        data.append(df)
        
    return data   

def extractFeatures(df):
    features = []
    
    features.append(np.var(df.values[:,0]))
    features.append(np.var(df.values[:,1])) 
    features.append(np.var(df.values[:,2]))
    features.append(np.var(df.values[:,3]))
    features.append(np.var(df.values[:,4]))
    features.append(np.argmax(df.values[:,0])) - np.argmin(df.values[:,0]) 
    features.append(np.argmax(df.values[:,1])) - np.argmin(df.values[:,1]) 
    features.append(np.argmax(df.values[:,2])) - np.argmin(df.values[:,2])
    features.append(np.argmax(df.values[:,3])) - np.argmin(df.values[:,3]) 
    features.append(np.argmax(df.values[:,4])) - np.argmin(df.values[:,4]) 
    features.append(df.values[0][6]) 
    return features

def getControlFeaure():
    control_data = []
    control = getFiles(control_path)
    for i in control:
        new_df = splitByTest(i)
        for k in range(new_df.shape[0]):
            last_df = np.array_split(i,70)
            for j in last_df:
                control_data.append(extractFeatures(j))
    return pd.Dataframe(control_data)            


In [13]:
df_control = getFiles(control_path)
#df_control
df_parkinson = getFiles(parkinson_path)
#df_parkinson